In [6]:
import pandas as pd
import numpy as np

stk_sheet = pd.read_excel('ship_stock_prices.xlsx', index_col=0, header=0)
stk_sheet


,Close,Open,High,Low,Volume
date,,,,,
2025.09.05,112500,115000,115000,110100,5328652
2025.09.04,118900,116100,120900,115500,2091765
2025.09.03,117700,123800,123800,116700,3438865
2025.09.02,123200,118200,123700,116700,4364919
2025.09.01,116400,112600,116600,111300,2631238
...,...,...,...,...,...
2021.08.12,30850,30400,31300,30400,292743
2021.08.11,30350,30500,31100,30200,265835
2021.08.10,30500,30850,31200,30200,357569


In [ ]:
def add_new_row(df_stock, df_add):
    df_new = df_stock.copy()
    new_row = df_add.index[0]   # '실질_GDP'

    # df_stock.columns(원래 날짜 문자열)을 분기 PeriodIndex 로 변환
    periods = pd.to_datetime(df_new.columns).to_period('Q')
    values = df_add.loc[new_row].reindex(periods).values
    df_new.loc[new_row] = values

    return df_new


def add_metric_row(df_stock, series, row_name):
    """
    df_stock: 열이 날짜 문자열로 된 DataFrame
    series: 날짜 문자열 인덱스의 pandas Series (e.g. PER or PBR)
    row_name: 새로 추가할 행 이름 (e.g. 'PER', 'PBR')
    """
    df_new = df_stock.copy()
    
    # 1) df_stock.columns → PeriodIndex('D')
    periods = pd.to_datetime(df_new.columns).to_period('D')
    
    # 2) series.index → PeriodIndex('D')
    ser = series.copy()
    ser.index = pd.to_datetime(ser.index).to_period('D')
    
    # 3) reindex → values
    values = ser.reindex(periods).values
    
    # 4) 새 행 추가
    df_new.loc[row_name] = values
    return df_new

# 기존에 GDP 행을 추가한 df_naver_add에 PER, PBR도 같은 기준으로 추가
df_naver_add = add_new_row(df_naver, df_GDP)        # 실질_GDP
df_naver_add = add_metric_row(df_naver_add, PER, 'PER')
df_naver_add = add_metric_row(df_naver_add, PBR, 'PBR')
df_naver_add = add_metric_row(df_naver_add, EPS, 'EPS')
df_naver_add = df_naver_add.T
df_naver_add.index.name = 'date'
df_naver_add.index

df_naver_add[df_naver_add[['PER', 'PBR', 'EPS']].isnull().any(axis=1)]

In [ ]:
## 기술적 지표 ##
# 추가할 window sizes
window_sizes = [5, 10, 20, 30, 60]

# 기존 변수명 유지하며 추가 지표 계산
for window in window_sizes:
    df_naver_add[f'SMA_{window}'] = df_naver_add['Close'].rolling(window=window).mean()
    df_naver_add[f'EMA_{window}'] = df_naver_add['Close'].ewm(span=window, adjust=False).mean()
    
    delta = df_naver_add['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    
    rs = avg_gain / avg_loss
    df_naver_add[f'RSI_{window}'] = 100 - (100 / (1 + rs))

# 결측값 채운건데 start date 이전의 데이터를 기반으로 계산하는 방법도 있음   
df_naver_add.interpolate(method='bfill', inplace=True)

df_naver_add.to_csv("dataset/stock/df_naver_add.csv")
print(df_naver_add.isnull().sum())

import matplotlib.pyplot as plt

# 그릴 컬럼명을 리스트에 정의
col_list = ['Close','EMA_5', 'SMA_5', 'EMA_60','SMA_60']

# x축: 날짜 인덱스
x = df_naver_add.index

plt.figure(figsize=(12, 6))
for col in col_list:
    plt.plot(x, df_naver_add[col], label=col)

plt.title('Close/EMA/SMA Window Sizes Overlay - naver', fontsize=20)
plt.xlabel('Date', fontsize=15)
plt.ylabel('Value', fontsize=15)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

import matplotlib.pyplot as plt

# 그릴 컬럼명을 리스트에 정의
col_list = ['RSI_20', 'RSI_60']

# x축: 날짜 인덱스
x = df_naver_add.index

plt.figure(figsize=(12, 6))
for col in col_list:
    plt.plot(x, df_naver_add[col], label=col)

plt.title('RSI Window Sizes Overlay - naver', fontsize=20)
plt.xlabel('Date', fontsize=15)
plt.ylabel('RSI Value', fontsize=15)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
PER = skt_sheet['PER']
PBR = skt_sheet['PBR']
EPS = skt_sheet['EPS']

price_dates = pd.to_datetime(df_naver.columns)
fund_dates  = pd.to_datetime(PER.index)
common_dates = price_dates.intersection(fund_dates).sort_values()

In [ ]:
## (PER,PBR) 2015-08-14가 결측임
# PER
pre  = df_naver_add.loc['2015-08-13','PER']
nxt  = df_naver_add.loc['2015-08-17','PER']
df_naver_add.loc['2015-08-14','PER'] = (pre + nxt) / 2

# PBR
pre2 = df_naver_add.loc['2015-08-13','PBR']
nxt2 = df_naver_add.loc['2015-08-17','PBR']
df_naver_add.loc['2015-08-14','PBR'] = (pre2 + nxt2) / 2

# EPS
pre3 = df_naver_add.loc['2015-08-13','EPS']
nxt3 = df_naver_add.loc['2015-08-17','EPS']
df_naver_add.loc['2015-08-14','EPS'] = (pre3 + nxt3) / 2